<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Combined_OpenAI_and_Elysia_Code_(as_provided).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## https://github.com/weaviate/elysia

In [ ]:
!pip install openai -q
!pip install elysia-ai -q

In [ ]:
from elysia import tool, Tree

In [ ]:
from google.colab import userdata
from openai import OpenAI
import asyncio
import os
import elysia

# Assuming these are available or need simple definitions based on documentation context
# Placeholder definition for read_response based on test usage
import json
def read_response(response):
    # This is a simplified placeholder based on how it's used in the test.
    # The actual response object from initialise_user/new_user_config/save_config_user
    # in a non-test environment might be different.
    try:
        # Check if response has a .body attribute and if it's bytes, decode it
        if hasattr(response, 'body') and isinstance(response.body, bytes):
             return json.loads(response.body.decode('utf-8'))
        # If it's already a dictionary or other JSON-serializable object
        elif isinstance(response, dict):
            return response
        # Fallback for other types
        return response
    except Exception as e:
        print(f"Error reading response body: {e}")
        # Fallback if response is not a JSONResponse or doesn't have .body
        return response # Or handle differently based on actual API return type


# Placeholder for get_user_manager based on test usage - this is highly dependent
# on Elysia's internal setup and might not have a simple equivalent in a user script.
# We'll try a basic approach first based on the test structure.
# If this fails, it indicates we need more info from the public API docs
# on how to obtain or initialize the user manager in a user script.
try:
    from elysia.api.dependencies.common import get_user_manager
    # Attempt to get the user manager if the import works
    user_manager_instance = get_user_manager()
except ImportError:
    print("Could not import get_user_manager. User manager setup might be different in user scripts.")
    user_manager_instance = None # Indicate failure


# --- Your OpenAI Code Block ---
api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

try:
    # Correcting the method and model name
    response = client.chat.completions.create(
        model="gpt-5", # Using a publicly available model
        messages=[{"role": "user", "content": "Write a one-sentence bedtime story about a unicorn."}]
    )
    # Correcting the attribute to access the response text
    print(response.choices[0].message.content)
except Exception as e:
    print(f"Error in OpenAI section: {e}")


# --- Your Elysia Code Block ---
# Implement the user configuration workflow based on documentation examples

async def configure_elysia_user(user_id: str, base_model: str, base_provider: str, user_manager):
    if user_manager is None:
        print("User manager is not available. Cannot configure Elysia user.")
        return None

    print(f"\nConfiguring Elysia user: {user_id}")
    try:
        # Import necessary functions from Elysia API routes based on documentation examples
        from elysia.api.routes.init import initialise_user
        from elysia.api.routes.user_config import new_user_config, save_config_user, load_a_config
        from elysia.api.api_types import SaveConfigUserData # Import SaveConfigUserData

        # Initialise the user
        init_response = await initialise_user(user_id=user_id, user_manager=user_manager)
        init_data = read_response(init_response)
        #print(f"Initialise user response: {init_data}")

        # Create a new config
        new_config_response = await new_user_config(user_id=user_id, user_manager=user_manager)
        new_config_data = read_response(new_config_response)
        config_id = new_config_data["config"]["id"]
        #print(f"New config created with ID: {config_id}")

        # Modify the config to set the base model and provider
        current_config = new_config_data["config"]
        current_config["settings"]["BASE_MODEL"] = base_model
        current_config["settings"]["BASE_PROVIDER"] = base_provider
        # Assuming COMPLEX_MODEL and COMPLEX_PROVIDER might also be needed or good practice
        current_config["settings"]["COMPLEX_MODEL"] = base_model
        current_config["settings"]["COMPLEX_PROVIDER"] = base_provider

        # Prepare frontend_config - Use empty dict as in one test example, and ensure Weaviate save is False
        # Add dummy values for WCD URL/API key to bypass check in load_a_config
        frontend_config_data = {
             "save_trees_to_weaviate": False,
             "save_configs_to_weaviate": False,
             "save_location_wcd_url": "dummy_url", # Provide dummy strings
             "save_location_wcd_api_key": "dummy_key", # Provide dummy strings
             "client_timeout": 3, # Default values from test
             "tree_timeout": 10,  # Default values from test
        }


        # Save the modified config
        # Need to provide SaveConfigUserData structure as seen in test
        save_data = SaveConfigUserData(
            name=current_config["name"], # Use the default name from the new config
            default=True, # Set as default config
            config=current_config,
            frontend_config=frontend_config_data, # Use the prepared frontend_config
        )

        save_response = await save_config_user(
            user_id=user_id,
            config_id=config_id,
            data=save_data,
            user_manager=user_manager,
        )
        save_data_response = read_response(save_response)
        #print(f"Config saved: {save_data_response}")

        # Load the config to confirm settings (optional, but good for verification)
        # Only attempt to load if save seemed successful and did not try Weaviate
        if 'warnings' in save_data_response and "Config has not been saved to Weaviate." in ''.join(save_data_response['warnings']):
             print("Skipping config load after save due to Weaviate save warning.")
        else:
             load_response = await load_a_config(
                 user_id=user_id, config_id=config_id, user_manager=user_manager
             )
             load_data_response = read_response(load_response)
             #print(f"Config loaded after save: {load_data_response}")
             # Safely access settings, checking if config key exists and then settings key
             if 'config' in load_data_response and 'settings' in load_data_response['config']:
                print(f"Loaded BASE_MODEL: {load_data_response['config']['settings'].get('BASE_MODEL')}")
             else:
                 print("Could not load config settings after save.")


        print("Elysia user configured successfully.")
        return user_manager # Return the user manager if successful

    except ImportError as e:
         print(f"Error importing Elysia API components needed for configuration: {e}")
         print("Please ensure elysia.api.routes.init, elysia.api.routes.user_config, and elysia.api.api_types are accessible.")
         return None
    except Exception as e:
        print(f"Error during Elysia user configuration: {e}")
        # Print detailed error info if available
        import traceback
        traceback.print_exc()
        return None


# Ensure the OpenAI API key is set as an environment variable for Elysia
# if it expects it that way, which the default OpenAI client does.
# We already retrieved it for the first part of your code.
try:
    os.environ["OPENAI_API_KEY"] = api_key
except TypeError: # api_key might be None if userdata.get failed
    print("API key not available for setting environment variable.")

# Explicitly set the BASE_MODEL environment variable for Elysia - Keeping this for now as it might still be relevant
# Although configuration via user settings is primary, environment variables might act as fallback or initial settings.
os.environ["BASE_MODEL"] = "gpt-4o" # Set the base model explicitly for Elysia

# Remove the import and usage of Config as it's causing an ImportError
# from elysia.config import Config # Removed
# Config.BASE_MODEL = "gpt-4o" # Removed

# Remove the unexpected 'client' and 'base_model'/'model' arguments from Tree initialization
# tree = Tree() # Removed previous initialization

# Need to perform configuration before initializing Tree
# Define a user ID for configuration
elysia_user_id = "colab_user" # Using a generic user ID for this environment


# Run the configuration process
configured_user_manager = await configure_elysia_user(
    user_id=elysia_user_id,
    base_model="gpt-5", # Specify the model you want to use
    base_provider="openai", # Specify the provider
    user_manager=user_manager_instance # Pass the potentially available user manager
)

# Initialize the Tree *after* configuration, potentially passing the configured user manager
# The documentation would ideally specify how Tree links to the user session.
# Assuming Tree might implicitly use the configured user manager if available globally or via context.
# If this still fails, the documentation is needed to know how to link Tree to the config.
if configured_user_manager:
    print("\nInitializing Tree after configuration...")
    # Try initializing Tree without args first, hoping it picks up the configured user manager or config
    # Based on the error from load_a_config about WCD URL/API key, it seems load_a_config might be
    # implicitly called by Tree() if a user_manager is available or if it tries to load a default config.
    # If Tree() still fails after configuration, we might need to explore how Tree is supposed to
    # be linked to the user_manager or the configured settings. For now, keep the simplest Tree() call.

    # Add the smart_setup call before initializing the Tree
    print("Running elysia.config.settings.smart_setup()...")
    try:
        elysia.config.settings.smart_setup()
        print("smart_setup completed.")
    except Exception as e:
        print(f"Error during smart_setup: {e}")
        # If smart_setup fails, the Tree initialization might also fail,
        # so we might want to skip the rest.
        configured_user_manager = None # Indicate failure


In [10]:
if configured_user_manager:
    tree = Tree()

    @tool(tree=tree)
    async def add(x: int, y: int) -> int:
        """
        Adds two integers together.
        """
        #print(f"DEBUG: add tool called with {x} and {y}")
        return x + y

    # Now run the Elysia part that uses the configured Tree
    # Change to synchronous run() based on test code
    def run_elysia_part_with_configured_tree_sync():
         #print("\nRunning Elysia part with configured Tree (synchronous)...")
         try:
            # tree.run is synchronous
            result = tree.run(
                "What is the sum of 9009 and 6006?",
            )
            # The run method might return the result differently, need to check its return type
            # Assuming it might return the final response text directly or in a specific structure
            print(f"Elysia result: {result}")
         except Exception as e:
            print(f"Error running Elysia part after configuration: {e}")

else:
    print("\nSkipping Elysia Tree execution due to configuration failure.")

In [11]:
# Call the synchronous run function
run_elysia_part_with_configured_tree_sync()

[08/13/25 08:56:28] WARNING  WCD_URL and WCD_API_KEY are not set. All Weaviate functionality will be  ]8;id=57977;file:///usr/local/lib/python3.11/dist-packages/elysia/util/client.py\client.py]8;;\:]8;id=582899;file:///usr/local/lib/python3.11/dist-packages/elysia/util/client.py#151\151]8;;\
                             disabled.                                                                             

╭─────────── User prompt ───────────╮
│                                   │
│ What is the sum of 9009 and 6006? │
│                                   │
╰───────────────────────────────────╯

╭──────────────── Assistant response ────────────────╮
│                                                    │
│ I am calculating the sum of 9009 and 6006 for you. │
│                                                    │
╰────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Current Decision ────────────────────────────────────────────────╮
│                                                                                                                 │
│ Node: base                                                                                                      │
│ Decision: add                                                                                                   │
│ Reasoning: The user's prompt is a straightforward arithmetic question asking for the sum of two integers, 9009  │
│ and 6006. The available action "add" is specifically designed to handle this type of task, as it adds two       │
│ integers together. This action directly addresses the user's request and will provide the correct sum of the    │
│ two numbers.                                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Current Decision ────────────────────────────────────────────────╮
│                                                                                                                 │
│ Node: base                                                                                                      │
│ Decision: text_response                                                                                         │
│ Reasoning: The user's prompt is a simple arithmetic question asking for the sum of two integers, 9009 and 6006. │
│ The "add" function is specifically designed to handle this task, and it has already been executed successfully, │
│ as indicated by the environment data showing the result of 15015. Therefore, the task has been completed, and I │
│ can now provide the user with the answer.                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────── Assistant response ────────╮
│                                    │
│ The sum of 9009 and 6006 is 15015. │
│                                    │
╰────────────────────────────────────╯

Elysia result: ('I am calculating the sum of 9009 and 6006 for you. The sum of 9009 and 6006 is 15015.', [[{'tool_result': 15015, '_REF_ID': 'add_default_0_0'}]])
